In [ ]:
# Copyright 2025 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### State 처리 분석

In [ ]:
import os
import time
from dotenv import load_dotenv
import argparse

from google.genai import types
from google.adk.runners import Runner
from google.adk.events import Event, EventActions

from google.adk.sessions import BaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import VertexAiSessionService

from agent import root_agent

In [ ]:
# 환경 변수 확인

# from dotenv import dotenv_values
# config = dotenv_values("../../.env")
# for key, value in config.items():
#     print(f"{key}: {value}")

In [ ]:
APP_NAME = "AI_assistant"
USER_ID = "Forusone"

session_service = InMemorySessionService()


In [ ]:
# 초기 상태 정의
init_state = {
    "task_status": "아직 답변 없음",
    "timestamp": time.time(),
}

In [ ]:
session = await session_service.create_session(
    app_name= APP_NAME,
    user_id=USER_ID,
    state=init_state
)
print(f"\n\n1. 초기 세션 상태: {session.state}")


In [ ]:
runner = Runner(agent=root_agent,
                app_name=APP_NAME,
                session_service=session_service)

In [ ]:
print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
    print("\n👤 User :   ")

    # 사용자 입력에서 쿼리 가져오기
    query = input("")
    if query.strip().lower() in ["exit", "quit"]:
        break
    else:
        print(query)
    
    content = types.Content(role='user', parts=[types.Part(text=query)])

    # 에이전트 러너를 사용하여 에이전트 이벤트를 비동기적으로 실행합니다.
    # 세션 정보는 user_id와 session_id를 통해 전달됩니다.
    events = runner.run_async(user_id=session.user_id,
                            session_id=session.id,
                            new_message=content,)

    async for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text            
            print("\n 🤖 AI Assistant: " + final_response)


    updated_session = await session_service.get_session(app_name = session.app_name, 
                                                     user_id = session.user_id, 
                                                     session_id = session.id)

    # 업데이트된 세션 상태에서 'last_turn' 표시
    print(f"\n\n output_key - last_turn : {updated_session.state['last_turn']}")


In [ ]:
# 상태 변경 준비
state_changes = {
    "task_status": f"답변 완료 : {query}",
    "timestamp": time.time(),
    "last_turn": updated_session.state['last_turn']
}

system_event = Event(
    invocation_id = "change-state",
    author = "system", # 또는 'agent', 'tool' 등
    actions = EventActions(state_delta=state_changes),
    timestamp = time.time()
)

# 상태 변경 이벤트 추가
await session_service.append_event(session, system_event)

print("\n\n2. 새로운 명시적 state-delta 이벤트를 추가했습니다.")

In [ ]:

updated_session = await session_service.get_session(app_name=session.app_name,
                                            user_id=session.user_id, 
                                            session_id=session.id)

print(f"\n\n3. 이벤트를 보낸 후 세션 상태: {updated_session.state}")

